In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Read the data
train = pd.read_csv('train.csv')
train.head()

,id,x001,x002,x003,x004,x005,x006,x007,x008,x009,...,x757,x758,x759,x760,x761,x762,x763,x764,x765,y
0,0,9.681860e+10,6991.15,7.76,0.00380,5.378811e+09,0.31,266117.20,934577.0,14539.0,...,0.0007,297281012,0.13,5.0,5,2.0,8.5127,14.28,-0.75,5
1,1,3.304810e+09,13914.43,5.37,0.00015,1.652405e+09,0.00,11927742.92,1798051.0,1051272.0,...,0.1136,3320000000000,0.08,661.0,0,350.0,1.5700,160.12,NaN,1
2,2,3.218944e+10,3991.98,5.77,0.00010,2.476111e+09,0.00,774385.01,375738.0,144143.0,...,0.0029,100474819,0.39,39.0,2,18.0,9.6800,25.06,-0.49,11
3,3,1.288000e+10,15937.45,5.86,0.00020,2.146667e+09,0.00,6324375.16,1932094.0,10055.0,...,0.0000,348000000000,0.25,2.0,1,0.0,4.5316,117.76,1.64,1
4,4,3.063412e+10,3621.00,7.52,0.00060,1.392460e+09,0.21,169860.29,474253.0,17914.0,...,0.0005,109546590,0.11,11.0,1,3.0,16.2717,5.81,-0.42,5


In [2]:
train.shape

(5380, 767)

In [3]:
train.dropna(inplace=True)
train.shape

(2857, 767)

Removing NAs almost halved the dataset. Two options:
- continue as usual. performance will probably suffer.
- imputation:
    - mean
    - lin reg prediction imputation?
    - k-NN imputation

#### Attempting Subset Selection
With 765 features, best subset selection is infeasible. Try forward stepwise selection. Stepwise also fails.

In [4]:
import statsmodels.formula.api as smf
import time
import itertools
X = train.drop(['id', 'y'], axis=1)
#Function to develop a model based on all predictors in predictor_subset
def processSubset(predictor_subset):
    # Fit model on feature_set and calculate R-squared
    model = smf.ols('y~' + '+'.join(predictor_subset),data = train).fit()
    Rsquared = model.rsquared
    return {"model":model, "Rsquared":Rsquared}

#Function to select the best model amongst all models with 'k' predictors
def getBest_model(k):
    tic = time.time()
    results = []
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset((list(combo))))

    # Wrap everything up in a dataframe
    models = pd.DataFrame(results)

    # Choose the model with the highest RSS
    best_model = models.loc[models['Rsquared'].argmax()]
    
    toc = time.time()
    print("Processed", models.shape[0], "models on", k, "predictors in", (toc-tic), "seconds.")
    return best_model

def best_sub_plots():
    plt.figure(figsize=(20,10))
    plt.rcParams.update({'font.size': 18, 'lines.markersize': 10})

    # Set up a 2x2 grid so we can look at 4 plots at once
    plt.subplot(2, 2, 1)

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The argmax() function can be used to identify the location of the maximum point of a vector
    plt.plot(models_best["Rsquared"])
    plt.xlabel('# Predictors')
    plt.ylabel('Rsquared')

    # We will now plot a red dot to indicate the model with the largest adjusted R^2 statistic.
    # The argmax() function can be used to identify the location of the maximum point of a vector

    rsquared_adj = models_best.apply(lambda row: row[1].rsquared_adj, axis=1)

    plt.subplot(2, 2, 2)
    plt.plot(rsquared_adj)
    plt.plot(1+rsquared_adj.argmax(), rsquared_adj.max(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('adjusted rsquared')

    # We'll do the same for AIC and BIC, this time looking for the models with the SMALLEST statistic
    aic = models_best.apply(lambda row: row[1].aic, axis=1)

    plt.subplot(2, 2, 3)
    plt.plot(aic)
    plt.plot(1+aic.argmin(), aic.min(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('AIC')

    bic = models_best.apply(lambda row: row[1].bic, axis=1)

    plt.subplot(2, 2, 4)
    plt.plot(bic)
    plt.plot(1+bic.argmin(), bic.min(), "or")
    plt.xlabel('# Predictors')
    plt.ylabel('BIC')
# best_sub_plots()

#Function to find the best predictor out of p-k predictors and add it to the model containing the k predictors
def forward(predictors):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p]))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['Rsquared'].argmax()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

def forward_selection():
    models_best = pd.DataFrame(columns=["Rsquared", "model"])

    tic = time.time()
    predictors = []

    for i in range(1,len(X.columns)+1):    
        models_best.loc[i] = forward(predictors)
        predictors = list(models_best.loc[i]["model"].params.index[1:])

    toc = time.time()
    print("Total elapsed time:", (toc-tic), "seconds.")
    return models_best

In [5]:
models_best = forward_selection()
best_sub_plots()

Processed  765 models on 1 predictors in 129.7945110797882 seconds.


RecursionError: maximum recursion depth exceeded while calling a Python object

#### Finding features with highest R-squared

In [6]:
# calculate R-squared for all individual features
r2 = {}
for feature in X:
    model = smf.ols('y~' + feature, data=train).fit()
    r2[feature] = model.rsquared

# sort the features by R-squared
r2 = pd.Series(r2)
r2.sort_values(ascending=False)

x689    1.000000
x117    1.000000
x320    1.000000
x379    1.000000
x039    1.000000
          ...   
x333   -0.579266
x509   -0.583145
x238   -0.583429
x667   -0.583855
x527   -0.585407
Length: 765, dtype: float64

In [9]:
# build a lin reg model with the features with r-squared = 1
r2_high = r2[r2==1]
r2_high.index

Index(['x039', 'x117', 'x320', 'x379', 'x689'], dtype='object')

In [ ]:
model = smf.ols('y~x039+x117+x320+x379+x689', data=train).fit()
model.summary()

Ridiculous output, probably need to scale features

In [17]:
# scale features
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = X.columns)
X_scaled.head()

,x001,x002,x003,x004,x005,x006,x007,x008,x009,x010,...,x756,x757,x758,x759,x760,x761,x762,x763,x764,x765
0,0.668222,-0.677898,1.775707,-0.081404,1.322632,0.041271,-0.595698,-0.667711,-0.297621,-0.257327,...,3.240399,-0.240519,-0.399543,-0.754507,-0.303890,0.926980,-0.299162,-0.474520,-0.864486,-0.400726
1,-0.469268,-0.924118,-0.882395,-0.331704,-0.267051,-2.150127,-0.526268,-0.994478,-0.177189,-0.257127,...,0.327782,-0.221789,-0.399550,0.157349,-0.194006,-0.215549,-0.206868,-0.394021,-0.749468,-0.336117
2,-0.496642,-0.954574,1.455132,-0.297880,-0.860520,-0.665632,-0.608847,-0.936874,-0.294485,-0.257365,...,-0.845643,-0.242222,-0.399550,-0.824650,-0.284499,-0.596392,-0.293393,0.060556,-0.954857,-0.318722
3,-0.265312,1.028477,-0.548463,0.003156,1.373784,1.384386,0.842849,1.417717,6.694100,0.166315,...,-0.871824,5.051465,5.403552,0.613276,5.765620,-0.596392,5.774928,-0.747451,1.883572,-0.060283
4,-0.757068,0.148774,-0.935824,-0.152436,-0.384045,1.313696,0.754728,0.108654,4.230821,-0.015003,...,2.542418,3.695266,-0.129371,1.805702,6.166375,-0.596392,6.109494,-0.676476,0.455128,-0.447941


In [21]:
print(y)

0        5
2       11
4        5
5        1
7        3
        ..
5365     2
5367    13
5370     4
5376     8
5378    13
Name: y, Length: 2857, dtype: int64


In [22]:
print(X_scaled)

          x001      x002      x003      x004      x005      x006      x007  \
0     0.668222 -0.677898  1.775707 -0.081404  1.322632  0.041271 -0.595698   
1    -0.469268 -0.924118 -0.882395 -0.331704 -0.267051 -2.150127 -0.526268   
2    -0.496642 -0.954574  1.455132 -0.297880 -0.860520 -0.665632 -0.608847   
3    -0.265312  1.028477 -0.548463  0.003156  1.373784  1.384386  0.842849   
4    -0.757068  0.148774 -0.935824 -0.152436 -0.384045  1.313696  0.754728   
...        ...       ...       ...       ...       ...       ...       ...   
2852 -0.331428 -0.679122  1.481846 -0.264056 -0.527218  0.394722 -0.548272   
2853 -0.902650  1.704555 -1.216327 -0.291115 -0.794420  0.677483  2.218174   
2854 -0.297292  0.199909  0.600265 -0.284350 -0.528823 -0.100109 -0.202803   
2855  0.597320  0.519209  0.707123 -0.257291  0.070795 -0.029419 -0.270168   
2856 -0.270004 -0.780691  1.829136 -0.294498 -0.901015 -0.100109 -0.612366   

          x008      x009      x010  ...      x756      x757    

In [24]:
y = train['y'].reset_index(drop=True)
y.head()

0     5
1    11
2     5
3     1
4     3
Name: y, dtype: int64

In [25]:
X_rel = X_scaled[['x039', 'x117', 'x320', 'x379', 'x689']]
X_rel.head()

,x039,x117,x320,x379,x689
0,-0.095279,0.833156,0.954093,0.831803,0.695762
1,-0.095279,-0.374565,-0.361891,-0.325758,-0.336024
2,-0.095279,-0.442889,-0.447593,-0.367489,-0.367303
3,-0.082036,0.016015,0.010564,-0.038195,-0.034525
4,-0.087657,-0.431577,-0.434333,-0.359483,-0.360269


In [26]:
import statsmodels.api as sm
# train a model with the 5 features with r-squared = 1, using scaled features
model = sm.OLS(y, X_rel['x039']).fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.000
Model:                            OLS   Adj. R-squared (uncentered):              0.000
Method:                 Least Squares   F-statistic:                              1.034
Date:                Fri, 21 Apr 2023   Prob (F-statistic):                       0.309
Time:                        16:34:55   Log-Likelihood:                         -11442.
No. Observations:                2857   AIC:                                  2.289e+04
Df Residuals:                    2856   BIC:                                  2.289e+04
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
x039          -0.2526      0.248     -1.017      0.309      -0.740       0.234
==============================================================================
Omnibus:                     2657.163   Durbin-Watson:                   1.214
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           102884.406
Skew:                           4.468   Prob(JB):                         0.00
Kurtosis:                      31.007   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

The R-squared is now being returned as zero - need to scale features first and then identify the most promising features.